In [26]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils import to_categorical
from PIL import Image
import numpy as np
from numpy import asarray
import glob  
import cv2 
import os

TrainPlanets = np.zeros((100, 300, 300, 3))
TrainPlanetsResults = np.zeros((100, 1))
i = 0
j=0

for planet in ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto', 'Galaxy']:
    img_dir = "IMAGES/"+planet 
    data_path = os.path.join(img_dir,'*g') 
    files = glob.glob(data_path) 
 
    for f1 in files: 
        rz = np.array(Image.open(f1).resize((300,300)))
        TrainPlanets[i] = rz
        TrainPlanetsResults[i] = j
        i = i+1
    j = j+1



In [27]:
TrainPlanets = TrainPlanets

X_train = TrainPlanets
Y_train = TrainPlanetsResults

Y_train = to_categorical(Y_train)

X_train[1]

array([[[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       ...,

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 2

In [28]:
model = ResNet50(include_top=False, input_shape=(300, 300, 3))
# add new classifier layers
for layer in model.layers:
	layer.trainable = False
    


C:\Users\Rishabh Iyer\.conda\envs\TensorFlow\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [29]:
flat1 = Flatten()(model.get_layer('activation_49').output)
class1 = Dense(1080, activation='relu', input_shape=(204800,))(flat1)
output = Dense(10, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)

In [30]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 150, 150, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [31]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [32]:
model.fit(x = X_train, y = Y_train, epochs = 4, batch_size = 25)

Epoch 1/4
100/100 [==============================] - 97s 973ms/step - loss: 132.6072 - accuracy: 0.1900
Epoch 2/4
100/100 [==============================] - 134s 1s/step - loss: 49.0962 - accuracy: 0.7600
Epoch 3/4
100/100 [==============================] - 46s 459ms/step - loss: 9.5804 - accuracy: 0.9300
Epoch 4/4
100/100 [==============================] - 45s 451ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [33]:
model.evaluate(x = X_train, y = Y_train)

100/100 [==============================] - 39s 390ms/step


[27.12816162109375, 0.6000000238418579]

In [38]:
img = np.array(Image.open('IMAGES/Test.png').resize((300,300)))
x = np.expand_dims(img, axis=0)

print(model.predict(x).argmax())

4


In [39]:
from keras.models import model_from_json

In [40]:
model.save("modelnew.h5")
print("Saved model to disk")

Saved model to disk
